In [21]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.losses import BinaryCrossentropy

In [22]:
G_train_df      = pd.read_csv ('../data/preprocessed_data/G_train.csv')
T_train_df      = pd.read_csv ('../data/preprocessed_data/T_train.csv')
G_T_train_df    = pd.read_csv ('../data/preprocessed_data/balanced_G_T_train.csv')

G_cv_df         = pd.read_csv ('../data/preprocessed_data/G_cv.csv')
T_cv_df         = pd.read_csv ('../data/preprocessed_data/T_cv.csv')
G_T_cv_df       = pd.read_csv ('../data/preprocessed_data/G_T_cv.csv')


In [23]:
random_state = 13
frac = 0.1

G_train_df      = G_train_df.sample(frac = frac, random_state=random_state)
T_train_df      = T_train_df.sample(frac = frac, random_state=random_state)
G_T_train_df    = G_T_train_df.sample(frac = frac, random_state=random_state)

In [24]:
y_train = G_T_train_df.drop(columns = ['group ID','technique ID' ]).values
y_train.dtype

# G_train = sampled_G_train_df.drop(columns = ids)
G_train = G_train_df.drop(columns = 'group ID').values

# T_train = sampled_T_train_df.drop(columns = ids)
T_train = T_train_df.drop(columns = 'technique ID').values

In [25]:
G_train_tf = tf.convert_to_tensor(G_train)
T_train_tf = tf.convert_to_tensor(T_train)
y_train_tf = tf.convert_to_tensor(y_train)
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))

batch_size = 32


train_dataset = train_dataset.shuffle(buffer_size=len(G_train_tf))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [26]:
y_cv = G_T_cv_df.drop(columns = ['group ID','technique ID' ]).values
y_cv.dtype

# G_cv = sampled_G_cv_df.drop(columns = ids)
G_cv = G_cv_df.drop(columns = 'group ID').values

# T_cv = sampled_T_cv_df.drop(columns = ids)
T_cv = T_cv_df.drop(columns = 'technique ID').values

In [27]:
G_cv_tf = tf.convert_to_tensor(G_cv)
T_cv_tf = tf.convert_to_tensor(T_cv)
y_cv_tf = tf.convert_to_tensor(y_cv)
cv_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_cv_tf, 'input_Technique': T_cv_tf}, y_cv_tf))
cv_dataset = cv_dataset.batch(batch_size)

In [41]:
import sys
sys.path.append('../models') 

from model_v0_1 import ContentBasedFiltering

In [42]:
# input shapes config
num_G_features = G_train.shape[1]  # remove Group ID during training
num_T_features = T_train.shape[1]   # remove Movie ID during training

model1 = ContentBasedFiltering (num_G_features=num_G_features, num_T_features=num_T_features)

TypeError: ContentBasedFiltering.__init__() missing 4 required positional arguments: 'Group_NN_width', 'Group_NN_depth', 'Technique_NN_width', and 'Technique_NN_depth'

In [ ]:
epochs = 50
history = model1.train (
    train_dataset,
    cv_dataset,
    epochs= epochs
)

Epoch 1/50


ValueError: in user code:

    File "c:\Users\vuchi\anaconda3\envs\tfNote01\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\vuchi\anaconda3\envs\tfNote01\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\vuchi\anaconda3\envs\tfNote01\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\vuchi\anaconda3\envs\tfNote01\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\vuchi\anaconda3\envs\tfNote01\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\vuchi\anaconda3\envs\tfNote01\lib\site-packages\keras\engine\input_spec.py", line 197